## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-01-01-15-00 +0000,wsj,U.S. Envoy Steve Witkoff to Visit Gaza As Trum...,https://www.wsj.com/world/middle-east/u-s-envo...
1,2025-08-01-01-13-27 +0000,nyt,Trump Imposes 35% Tariff on Canada and Grants ...,https://www.nytimes.com/2025/07/31/us/politics...
2,2025-08-01-01-10-01 +0000,nyt,Andrea Lucas Confirmed to Equal Employment Opp...,https://www.nytimes.com/2025/07/31/us/politics...
3,2025-08-01-01-09-40 +0000,nyt,Trump to Impose Sweeping New Tariffs on Much o...,https://www.nytimes.com/2025/07/31/us/politics...
4,2025-08-01-01-06-00 +0000,wsj,"South Korea’s Exports Accelerate, Fueled by Ch...",https://www.wsj.com/economy/trade/south-koreas...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,78
28,tariffs,29
27,new,16
12,tariff,16
42,trade,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
40,2025-07-31-23-00-10 +0000,nypost,Trump struck huge trade deals — but tariffs ar...,https://nypost.com/2025/07/31/us-news/dozens-o...,191
183,2025-07-31-16-29-38 +0000,latimes,New Trump tariffs on Mexico delayed 90 days as...,https://www.latimes.com/world-nation/story/202...,153
136,2025-07-31-19-12-10 +0000,nyt,Appeals Court Hears Legal Challenge to Trump T...,https://www.nytimes.com/2025/07/31/business/tr...,147
9,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...,140
202,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...,140


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
40,191,2025-07-31-23-00-10 +0000,nypost,Trump struck huge trade deals — but tariffs ar...,https://nypost.com/2025/07/31/us-news/dozens-o...
9,62,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...
22,62,2025-07-31-23-44-33 +0000,nypost,White House releases new tariff rates for doze...,https://nypost.com/2025/07/31/us-news/white-ho...
281,54,2025-07-31-08-34-03 +0000,nypost,"Russian missile and drone attack batters Kyiv,...",https://nypost.com/2025/07/31/world-news/russi...
202,45,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...
315,45,2025-07-31-03-54-00 +0000,wsj,Senators rejected an effort to block U.S. arms...,https://www.wsj.com/politics/policy/republican...
224,42,2025-07-31-13-58-00 +0000,wsj,South Africa’s central bank cut its main repo ...,https://www.wsj.com/articles/south-african-res...
275,41,2025-07-31-09-14-50 +0000,nypost,Trump says it’s hard to make deal with Canada ...,https://nypost.com/2025/07/31/us-news/trump-sa...
324,35,2025-07-31-01-23-00 +0000,wsj,Fed Chair Jerome Powell is keeping his options...,https://www.wsj.com/economy/central-banking/po...
109,33,2025-07-31-20-45-00 +0000,wsj,President Trump’s popular policy will cut taxe...,https://www.wsj.com/politics/policy/no-tax-ove...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
